# Item-Based Collaborative Filtering - Pearson Correlation

## 1. Set-up
import dependent packages and declare consts

In [1]:
# package initialization

import pandas as pd
import numpy as np
from surprise import BaselineOnly, KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn import metrics

In [20]:
# consts

# DATASET = '../../data/extracted/LON-A/London_Attractions_Complete_Review.csv'
DATASET = '../../data/extracted/NYC-R/New_York_City_Restaurant_Complete_Review.csv'
OCCURENCE_THRESHOLD = 5

## 2. Read Dataset
read dataset in csv format into pandas.DataFrame

In [21]:
# read dataset

df = pd.read_csv(DATASET, sep='\t')

In [22]:
# print dataset information

print("Columns: \n", list(df.columns))
print("\nShape: \n", df.shape)

Columns: 
 ['Unnamed: 0', 'Unnamed: 0.1', 'rtime', 'rquote', 'iid', 'rrate', 'rid', 'uage', 'ucity', 'ucountry', 'ugender', 'uhometown', 'uid_index', 'ulevel', 'uname_y', 'usince', 'ustate', 'ustyle', 'iattribute', 'ilocality', 'iname', 'ipopularity', 'ipost', 'iprice', 'irating', 'iregion', 'istreet', 'itag']

Shape: 
 (129964, 28)


## 3. Data Preprocessing

* Retain users/items with at least five ratings only
* Data splitting
  - the latest 20% interactions (by time)
  - Randomly split the remaining data into training (70%) and validation (10%) sets
* Transform the ratings into binary implicit feedback as ground truth, indicating whether the user has interacted with the specific item

In [6]:
def sort_by_time(df):
    
    # here we use 'rid' for sorting becaz it's auto incrementing
    return df.sort_values(by=['rid'], ascending=True)

In [7]:
def filter_by_occurrence(df, column, threshold):
    return df.groupby(column).filter(lambda x: len(x) >= threshold)

In [8]:
def convert_binary(df):
    df.loc[df['rrate'] != "None", 'rrate'] = 1.0
    df.loc[df['rrate'] == "None", 'rrate'] = 0.0
    return df

In [9]:
def data_preprocess(dataframe):
    
    # sort by time (ascending order)
    df = sort_by_time(dataframe)
    
    # retrieve needed columns
    df = df[['uid_index', 'iid', 'rrate']]
    
    # convert ratings into binarys
    df = convert_binary(df)
    
    df['rrate'] = pd.to_numeric(df['rrate'])
    
    # Retain users/items with at least five ratings only
    df = filter_by_occurrence(df, 'iid', OCCURENCE_THRESHOLD)
    df = filter_by_occurrence(df, 'uid_index', OCCURENCE_THRESHOLD)
    
    # split dataset into training set, validation set and test set
    users = df.groupby('uid_index')
    
    test_df = pd.DataFrame()
    train_validation_df = pd.DataFrame()
    
    # for each user, get its latest 20% rating as test set
    for uid in users.size().to_dict().keys():
        user = users.get_group(uid)
        split_idx = int(len(user)*0.8)
        test_df = test_df.append(user.iloc[split_idx:])
        train_validation_df = train_validation_df.append(user.iloc[:split_idx])
    
    train_validation_df = train_validation_df.reindex(np.random.permutation(train_validation_df.index)) # shuffle
    train_df = train_validation_df.iloc[:int(len(train_validation_df)*0.875)]
    validation_df = train_validation_df.iloc[int(len(train_validation_df)*0.875):]
    
    return (train_df, validation_df, test_df)

In [23]:
# dataset preprocessing

train_df, validation_df, test_df = data_preprocess(df)
print("training set size: ", train_df.shape)
print("validation set size: ", validation_df.shape)
print("test set size: ", test_df.shape)

training set size:  (80722, 3)
validation set size:  (11532, 3)
test set size:  (29517, 3)


## 4. Load into Surprise

In [24]:
reader = Reader(rating_scale=(0, 1))
train_dataset = Dataset.load_from_df(train_df, reader).build_full_trainset()

## 5. Model declaration & Fitting

Collaborative Filtering - KNN:

\begin{equation}
\hat{r}_{ui} = \frac{
\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}
{\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}
\end{equation}

**Parameters**
- `k` – The max number of neighbors to take into account for aggregation, defaults to `40`

In [11]:
# compute pearson correlation between items
sim_options = {'name': 'pearson', 'user_based': False}

K_MAX = 40

In [12]:
algorithm = KNNBasic(k=K_MAX, sim_options=sim_options)

In [13]:
algorithm.fit(train_dataset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


## 6. Prediction

In [17]:
def predict(algorithm, dataframe):
    z = []
    y = []
    for i in range(len(dataframe)):
        user = dataframe.iloc[i][0]
        item = dataframe.iloc[i][1]
        rating = dataframe.iloc[i][2]
        prediction = algorithm.predict(user, item, r_ui=rating, verbose=False)
        
        if prediction.details['was_impossible'] == False:
            z.append(prediction.est)
            y.append(rating)
    return (np.array(z, dtype=np.float32), np.array(y, dtype=np.int))

In [15]:
validation_z, validation_y = predict(algorithm, validation_df)

In [16]:
test_z, test_y = predict(algorithm, test_df)

## 7. Evaluation

AUC metric

In [12]:
def evaluate_auc(z, y):
    return metrics.roc_auc_score(y, z)

In [18]:
print("validation AUC: ", evaluate_auc(validation_z, validation_y))
print("test AUC: ", evaluate_auc(test_z, test_y))

validation AUC:  0.6000936674651434
test AUC:  0.6828113448211428


LogLoss metric

In [13]:
# assume parameters z & y are ndarray
def evaluate_logloss(z, y):
    zz = np.ones((z.shape[0], 2))
    zz[:, 0] -= z
    zz[:, 1] = z
    return metrics.log_loss(y, zz)

In [20]:
print("validation LogLoss: ", evaluate_logloss(validation_z, validation_y))
print("test LogLoss: ", evaluate_logloss(test_z, test_y))

validation LogLoss:  11.700918064150127
test LogLoss:  9.947000461167546


NDCG metric

In [14]:
# assume parameters z & y are ndarray
def evaluate_ndcg(z, y):
    return metrics.ndcg_score(np.expand_dims(y, axis=0), np.expand_dims(z, axis=0), k=5)

In [22]:
print("validation NDCG@5: ", evaluate_ndcg(validation_z, validation_y))
print("test NDCG@5: ", evaluate_ndcg(test_z, test_y))

validation NDCG@5:  0.7739361702127657
test NDCG@5:  0.8534635879218472


## Experiments

In [15]:
def train_ks(start=3, end=20, step=1):

    history = []
    for k in range(start, end+step, step):
        
        print("Using K =", k)
        sim_options = {'name': 'cosine', 'user_based': False}
        algorithm = KNNBasic(k=k, sim_options=sim_options)
        
        algorithm.fit(train_dataset)
        validation_z, validation_y = predict(algorithm, validation_df)
        test_z, test_y = predict(algorithm, test_df)
        
        history.append({
            'k': k,
            'val_auc': evaluate_auc(validation_z, validation_y),
            'test_auc': evaluate_auc(test_z, test_y),
            'val_logloss': evaluate_logloss(validation_z, validation_y),
            'test_logloss': evaluate_logloss(test_z, test_y),
            'val_ndcg': evaluate_ndcg(validation_z, validation_y),
            'test_ndcg': evaluate_ndcg(test_z, test_y)
        })
    return history

In [27]:
history = train_ks(start=21, end=30, step=1)

Using K = 21
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 22
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 23
Computing the cosine similarity matrix...


...

Done computing similarity matrix.
Using K = 29
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Using K = 30
Computing the cosine similarity matrix...


E:\ProgramData\Anaconda3\envs\surprise\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.


In [28]:
print("| K | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |")
print("|:-- | -- | -- | -- | -- | -- | -- |")
for his in history:
    print("| k={} | {:.5f} | {:.5f} | {:.5f} | {:.5f} | {:.5f} | {:.5f} |".format(
        his['k'],
        his['val_auc'],
        his['val_logloss'],
        his['val_ndcg'],
        his['test_auc'],
        his['test_logloss'],
        his['test_ndcg'],
    ))

| K | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| k=21 | 0.96062 | 0.10630 | 0.99766 | 0.97319 | 0.08895 | 0.99837 |
| k=22 | 0.96087 | 0.10628 | 0.99766 | 0.97325 | 0.08894 | 0.99837 |
| k=23 | 0.96112 | 0.10624 | 0.99766 | 0.97344 | 0.08892 | 0.99837 |
| k=24 | 0.96112 | 0.10624 | 0.99766 | 0.97350 | 0.08891 | 0.99837 |
| k=25 | 0.96112 | 0.10622 | 0.99766 | 0.97350 | 0.08891 | 0.99837 |
| k=26 | 0.96112 | 0.10622 | 0.99766 | 0.97356 | 0.08890 | 0.99837 |
| k=27 | 0.96111 | 0.10622 | 0.99766 | 0.97356 | 0.08890 | 0.99837 |
| k=28 | 0.96111 | 0.10622 | 0.99766 | 0.97356 | 0.08890 | 0.99837 |
| k=29 | 0.96111 | 0.10622 | 0.99766 | 0.97362 | 0.08890 | 0.99837 |
| k=30 | 0.96111 | 0.10624 | 0.99766 | 0.97479 | 0.08644 | 0.99845 |


## Experiment Results

LON-A dataset:  

| settings | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| **k=3*** | 0.48745 | 2.95258 | 0.91355 | 0.50680 | 2.84786 | 0.91566 |
| k=4 | 0.47470 | 2.96913 | 0.90896 | 0.48928 | 2.86664 | 0.90873 |
| k=5 | 0.46689 | 2.99067 | 0.90563 | 0.47904 | 2.88459 | 0.90487 |
| k=6 | 0.45574 | 3.00328 | 0.90251 | 0.47158 | 2.88702 | 0.90186 |
| k=7 | 0.44457 | 3.02319 | 0.89955 | 0.46571 | 2.88416 | 0.89949 |
| k=8 | 0.44071 | 3.01225 | 0.89829 | 0.46105 | 2.87906 | 0.89776 |
| k=9 | 0.43548 | 3.01783 | 0.89643 | 0.45869 | 2.86922 | 0.89669 |
| k=10 | 0.43257 | 3.02031 | 0.89505 | 0.45468 | 2.87365 | 0.89523 |

NYC-R dataset:  

| settings | validation AUC | validation LogLoss | validation NDCG@5 | testing AUC | testing LogLoss | testing NDCG@5 |
|:-- | -- | -- | -- | -- | -- | -- |
| **k=3*** | 0.93551 | 0.09610 | 0.99781 | 0.96075 | 0.07931 | 0.99863 |
| k=4 | 0.94056 | 0.10807 | 0.99764 | 0.96564 | 0.08325 | 0.99858 |
| k=5 | 0.94329 | 0.10755 | 0.99765 | 0.96767 | 0.08523 | 0.99851 |
| k=6 | 0.94669 | 0.10750 | 0.99765 | 0.96545 | 0.09460 | 0.99829 |
| k=7 | 0.94935 | 0.10717 | 0.99766 | 0.96983 | 0.08710 | 0.99844 |
| k=8 | 0.95174 | 0.10693 | 0.99766 | 0.97063 | 0.08696 | 0.99844 |
| k=9 | 0.95284 | 0.10678 | 0.99766 | 0.97124 | 0.08692 | 0.99844 |
| k=10 | 0.95458 | 0.10665 | 0.99766 | 0.97154 | 0.08684 | 0.99844 |
| k=11 | 0.95491 | 0.10653 | 0.99766 | 0.97212 | 0.08672 | 0.99844 |
| k=12 | 0.95491 | 0.10651 | 0.99766 | 0.97240 | 0.08667 | 0.99844 |
| k=13 | 0.95584 | 0.10658 | 0.99766 | 0.97287 | 0.08668 | 0.99844 |
| k=14 | 0.95707 | 0.10647 | 0.99766 | 0.97307 | 0.08662 | 0.99845 |
| k=15 | 0.95794 | 0.10654 | 0.99766 | 0.97221 | 0.08907 | 0.99837 |
| k=16 | 0.95879 | 0.10652 | 0.99766 | 0.97255 | 0.08903 | 0.99837 |
| k=17 | 0.95905 | 0.10644 | 0.99766 | 0.97261 | 0.08902 | 0.99837 |
| k=18 | 0.95932 | 0.10642 | 0.99766 | 0.97281 | 0.08900 | 0.99837 |
| k=19 | 0.95932 | 0.10640 | 0.99766 | 0.97287 | 0.08898 | 0.99837 |
| k=20 | 0.95985 | 0.10636 | 0.99766 | 0.97306 | 0.08896 | 0.99837 |
| k=21 | 0.96062 | 0.10630 | 0.99766 | 0.97319 | 0.08895 | 0.99837 |
| k=22 | 0.96087 | 0.10628 | 0.99766 | 0.97325 | 0.08894 | 0.99837 |
| k=23 | 0.96112 | 0.10624 | 0.99766 | 0.97344 | 0.08892 | 0.99837 |
| k=24 | 0.96112 | 0.10624 | 0.99766 | 0.97350 | 0.08891 | 0.99837 |
| k=25 | 0.96112 | 0.10622 | 0.99766 | 0.97350 | 0.08891 | 0.99837 |
| **k=26*** | 0.96112 | 0.10622 | 0.99766 | 0.97356 | 0.08890 | 0.99837 |
| k=27 | 0.96111 | 0.10622 | 0.99766 | 0.97356 | 0.08890 | 0.99837 |
| k=28 | 0.96111 | 0.10622 | 0.99766 | 0.97356 | 0.08890 | 0.99837 |
| k=29 | 0.96111 | 0.10622 | 0.99766 | 0.97362 | 0.08890 | 0.99837 |
| k=30 | 0.96111 | 0.10624 | 0.99766 | 0.97479 | 0.08644 | 0.99845 |